# Modifying Data and Reporting with SQL
### Code Louisville, June 13, 2019

#### Dockless Vehicle (Scooter) Data
In this class, we will examine the concepts covered in the [Modifying Data with SQL](https://teamtreehouse.com/library/modifying-data-with-sql) and [Reporting with SQL](https://teamtreehouse.com/library/reporting-with-sql) Treehouse courses by working with the dockless vehicle data dataset published by the City of Louisville.

The dockless vehicle data page is https://data.louisvilleky.gov/dataset/dockless-vehicles
This page contains info about what data is in the CSV file we'll work with.

The data file we will be working with is the CSV file named [`Dockless Vehicle Trips - Block Level`](https://data.louisvilleky.gov/sites/default/files/DocklessTripOpenData_3.csv).

###### Step 1: Loading Data

In your Treehouse course, you learned about the `INSERT` statement.
This is a very useful statement for inserting rows one or many at a time manually when you want to write the SQL by hand.

But in this Code Louisville course, you will likely be working with a lot of data in the form of a CSV that you want to convert to a SQL table. Fortunately a library you've already learned about, `pandas` has a helpful functions for reading from a CSV and saving a DataFrame to a database:

In [51]:
import sqlite3
import pandas as pd

# Create a database file named scooter_data.db
conn = sqlite3.connect('scooter_data.db')

# Read the data into a pandas DataFrame
scooter_data = pd.read_csv('DocklessTripOpenData_3.csv')

# Save to a table named scooter_data
# (Columns will be named after the column headers in the CSV file)
# Read about DataFrame's to_sql method here: https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.to_sql.html
scooter_data.to_sql('scooter_data', conn, if_exists='replace')

##### Step 2: Modifying Data

The scooter data that we loaded into our database has most of the data we want. But what if we always wanted to use kilometers instead of miles when looking at the data?

From the city's [data page](https://data.louisvilleky.gov/dataset/dockless-vehicles), we can see that there is a column in the data that contains the trip distance in miles: `TripDistance - distance of trip in miles based on company route data`.

Since we want to work in kilometers instead of miles, let's modify the data in the database and change the values from miles to kilometers.

First, let's look at what a few rows look like before making the change:  
(*We will use ORDER BY and LIMIT so that we get the same results each time we run this query*)

In [52]:
pd.read_sql_query("""
    SELECT TripID, TripDistance FROM scooter_data
    ORDER BY TripID DESC
    LIMIT 5;
""", conn)

,TripID,TripDistance
0,ffff9a78-8f18-0ff1-cec5-c5a778a0,0.00
1,ffff30aa-b4c7-87d4-0c09-fbb7828f,0.08
2,fffdd4ed-b41a-78fa-7354-17213196,3.37
3,fffd9458-800a-c1ef-5de1-e8860f81,0.43
4,fffcde54-21d5-fbf8-6068-febc7ab4,1.32


Now, let's update the `TripDistance` value and then run the same query to see the new values:  
*(We only want to update distance values that are greater than 0)*

In [53]:
results = pd.io.sql.execute("""
    UPDATE scooter_data
    SET TripDistance = TripDistance * 1.60934
    WHERE TripDistance > 0;
""", conn)
conn.commit()

print('Updated {} rows'.format(results.rowcount))

pd.read_sql_query("""
    SELECT TripID, TripDistance FROM scooter_data
    ORDER BY TripID DESC
    LIMIT 5;
""", conn)

Updated 131314 rows


,TripID,TripDistance
0,ffff9a78-8f18-0ff1-cec5-c5a778a0,0.000000
1,ffff30aa-b4c7-87d4-0c09-fbb7828f,0.128747
2,fffdd4ed-b41a-78fa-7354-17213196,5.423476
3,fffd9458-800a-c1ef-5de1-e8860f81,0.692016
4,fffcde54-21d5-fbf8-6068-febc7ab4,2.124329


Comparing the output from the same query, we can see that the `TripDistance` values have increased by `1.60934`. Awesome!

##### Step 3: Deleting Data

From the city's [data page](https://data.louisvilleky.gov/dataset/dockless-vehicles), we can see that there is a `Data Outliers` section that talks about possible bad data.

There are two things that stand out:
```
  * Set TripDistance to -1 where the value is less than 0
  * Set TripDistance to 100 where the value is over 100 (maximum range is currently around 25 miles)
```

This means that any `TripDistance` values that are `-1` or `100` are probably bad data. We could try to remember to exclude these from all of our queries, but it's probably easier to delete these values from the data set. Lets's do that now!

In [54]:
results = pd.io.sql.execute("""
    DELETE FROM scooter_data
    WHERE TripDistance = -1 OR TripDistance = 100;
""", conn)
conn.commit()

print('Deleted {} rows'.format(results.rowcount))

Deleted 82 rows


##### Step 4: Aggregating Data

So what's an aggregation?  
The dictionary definition is: `a cluster of things that have come or been brought together`.

But how does this apply to rows in a database?  
Through aggregate functions that perform an operation on this "cluster of things" (aka grouping).

The aggregate functions provided by sqlite are:
  * avg
  * count
  * group_concat
  * max
  * min
  * sum
  
You'll most often use these functions with numerical values.

Let's try a few!

In [58]:
pd.read_sql_query("""
    SELECT 
        min(TripDistance) as 'Shortest Trip Distance',
        max(TripDistance) as 'Longest Trip Distance',
        avg(TripDistance) as 'Average Trip Distance',
        sum(TripDistance) as 'Total Trip Distance'
    FROM scooter_data
""", conn)

,Shortest Trip Distance,Longest Trip Distance,Average Trip Distance,Total Trip Distance
0,0.0,160.934,1.898254,272427.9036


Useful, but these values are across *all* rides in the table. What if we want to change the "cluster" from the one cluster comprised of the entire table into a cluster per day?

We can do that with the `GROUP BY` clause:

In [63]:
pd.read_sql_query("""
    SELECT 
        StartDate,
        min(TripDistance) as 'Shortest Trip Distance',
        max(TripDistance) as 'Longest Trip Distance',
        avg(TripDistance) as 'Average Trip Distance',
        sum(TripDistance) as 'Total Trip Distance'
    FROM scooter_data
    GROUP BY StartDate
    LIMIT 5; -- Limit to make it easier to read
""", conn)

,StartDate,Shortest Trip Distance,Longest Trip Distance,Average Trip Distance,Total Trip Distance
0,2018-08-09,0.000000,19.730508,2.802325,826.685771
1,2018-08-10,0.000000,14.130005,2.587120,1179.726687
2,2018-08-11,0.000000,16.302614,3.867068,989.969408
3,2018-08-12,0.016093,23.673391,3.928000,1253.032124
4,2018-08-13,0.000000,20.631739,3.786478,632.341873


So now we have the shortest, longest, average, and total trip distances *per day*.  
Pretty useful!

*Note: if you use the `GROUP BY` clause, the values you return in your select statement either have to be aggregates or columns included in the group by clause*

Now that we know how to use aggregates, let's try to answer a question.

What if someone asked you: **what days of the week are most popular for scooter rides?**  
Could we write a query to answer that? Yes, we can!

In [69]:
pd.read_sql_query("""
    SELECT 
        DayOfWeek,
        count(*) as 'Total Rides'
    FROM scooter_data
    GROUP BY DayOfWeek
    ORDER BY count(*) DESC
""", conn)

,DayOfWeek,Total Rides
0,7,25669
1,6,21953
2,5,20114
3,3,19956
4,4,19335
5,1,18973
6,2,17515


Based on this decription from the city's [data page](https://data.louisvilleky.gov/dataset/dockless-vehicles):
`DayOfWeek - 1-7 based on date, 1 = Sunday through 7 = Saturday, useful for analysis`,
we can say that Saturday is the most popular day for scooter rides, followed by Friday. The least popular day for scooter rides is Monday.

##### Conclusion
Modifying data and reporting with SQL is a powerful way to organize and explore your data.  
Understanding how to manipulate and query data in a SQL database is important in a variety of programming tasks. You can answer a lot of questions with just SQL.

In future Treehouse courses, you'll learn how to use additional Python libraries to perform some aggregations outside of SQL and generate visualizations. 